# Quality Control and Trimming

This notebook will go through the workflow for read quality control and trimming. 

-----------

Sections:

1. Write the run script to check the quality of the reads BEFORE trimming (using fastqc)
2. Write the run script to trim and filter low-quality reads with [Trimmomatic](https://carpentries-lab.github.io/metagenomics-analysis/03-trimming-filtering/index.html).
3. Write the run script to check the quality of the reads AFTER trimming (using fastqc)
4. Launch each of the run scripts using the launcher script.

-----------

Time to run: Once you launch these scripts, it will take approximately 1-2 hours to run.


## Getting Started

Before we get started you will need to set several variables that we will use throughout this notebook. 

In [ ]:
# set the variables for your name and accessions
# name is your output directory
# accessions is a list of SRA accession ids, or your sample ids.
id = "MY_ID"
accessions = "MY_ACCESSIONS"

In [ ]:
# Go into the working directory
work_dir = "/my_dir_path/" + id + "/01_qc_trimming"
%cd $work_dir

In [ ]:
# Set the fastq directory. This is where we have our raw fastq files.
fastq_dir = "/my_dir_path/" + id + "/00_getting_data"

## Creating a config file
Each of the run scripts below executes code that requires certain variables to be set. So we don't need to edit the code in each of the scripts, we are going to use a config file that defines all of these variables. Then when we want to use these variables in the script, we will "source" the config file to set the variables. This is generally a good practice in writing scripts on the HPC, that makes it so you only need to modify the config file (rather than each individual run scripts). 

In [ ]:
# create a config file with all of the variables you need
!echo "export ID=$id" > config.sh
!echo "export ACCESSIONS=$accessions" >> config.sh
!echo "export FASTQC=/path_to_containers/fastqc-0.11.9.sif" >> config.sh
!echo "export TRIMMOMATIC=/path_to_containers/trimmomatic:0.39--hdfd78af_2.sif" >> config.sh
!echo "export WORK_DIR=$work_dir" >> config.sh
!echo "export FASTQ_DIR=$fastq_dir" >> config.sh

### Step 1: Assessing Read Quality for the Raw Reads

Now that we have all of our raw sequence data downloaded, we are ready to start the quality control process. We will use a tool called fastqc that generates a report about the quality of our sequence data. First, we create reports showing us the quality of the reads from each accession BEFORE trimming. That way we can see how well our trimming step works.

Let's create a run script to run the fastqc program.

In [ ]:
# Create a script to run fastqc on each of our accessions
# A few important points:
# 1. We are using the variables from the config file via the `source ./config.sh` command in the script.
# 2. fastqc runs on each of the fastq files in the $FASTQ_DIR
# 3. We will copy the reports to our home directory to visualize these results (via ondemand Jupyter)
# 4. Array is the number of samples, counting from zero

my_code = '''#!/bin/bash
# --------------------------------------------------
# Request resources here
# --------------------------------------------------
#SBATCH --job-name=fastqc               # job name
#SBATCH --ntasks=1                      # number of CPUs required 
#SBATCH --cpus-per-task=1               # number of CPU cores per task 
#SBATCH --nodes=1                       # number of nodes
#SBATCH --mem-per-cpu=4000              # memory per CPU core in MB (see also --mem) 
#SBATCH --time=10:00:00                 # time limit hrs:min:sec
#SBATCH --partition=standard            # partition name (i.e. standard, windfall)
#SBATCH --account=your_account          # account name (name of your group)                     
#SBATCH --output=process-%j.out         # standard output file name (%j expands to jobID)
#SBATCH --error=process-%j.err          # standard error file name (%j expands to jobID)                      

# --------------------------------------------------
# Load modules here
# --------------------------------------------------

# --------------------------------------------------
# Execute commands here
# --------------------------------------------------

# echo for log
echo "job started"; pwd; hostname; date

# source config gile and get sample name from input list based on job array index
source ./config.sh
export SAMPLE=`head -n +${SLURM_ARRAY_TASK_ID} $IN_LIST | tail -n 1`

# run FastQC
echo "Running FastQC on sample $SAMPLE"                       # print sample name to log
apptainer run ${FASTQC} fastqc $FASTQ_DIR/${SAMPLE}_*.fastq*

# move FastQC results to a separate directory and copy to local directory for viewing
# create directory if it does not exist
TRIM_DIR="${WORK_DIR}/before_qc_trimming"
if [[ ! -d "$TRIM_DIR" ]]; then
  echo "$TRIM_DIR does not exist. Directory created"
  mkdir -p $TRIM_DIR
fi
#  move files
mv $FASTQ_DIR/${SAMPLE}_*_fastqc.html $TRIM_DIR
mv $FASTQ_DIR/${SAMPLE}_*_fastqc.zip $TRIM_DIR
cp -r $TRIM_DIR ~/01_qc_trimming

# print date for log
echo "job ended"; date
 
'''

with open('01A_run_fastqc-slurm.sh', mode='w') as file:
    file.write(my_code)

### Step 2: Creating a run script to trim and filter bad reads from the .fastq files

In order to run trimmomatic in a PE (paired-end) format we'll need two files. These files are:  *_1.fastq.gz and *_2.fastq.gz for each accession from the SRA (or your sequencing rrun). You downloaded these in 00_getting_data. 

### Initial Data Management

Trimmomatic will give us 4 output files (forward paired, forward unpaired, reverse paired and reverse unpaired. To keep our data organized, let's create output directories so the script can organize our data as it runs.


In [ ]:
# Create the trimmed and unpaired directories
import os

trim_dir = work_dir + "/trimmed_reads"
unpair_dir = work_dir + "/unpaired_reads"

if os.path.isdir(trim_dir):
    print("trim_dir exists")
else:
    os.mkdir(trim_dir)

if os.path.isdir(unpair_dir):
    print("unpair_dir exists")
else:
    os.mkdir(unpair_dir)

In [ ]:
# Let's create a run script that runs trimmomatic on all of our fastq files
# you can only run this after the *.fastq files are gzipped check in your 00_getting_data directory

my_code = '''#!/bin/bash
# --------------------------------------------------
# Request resources here
# --------------------------------------------------
#SBATCH --job-name=trimmomatic          # job name
#SBATCH --ntasks=1                      # number of CPUs required 
#SBATCH --cpus-per-task=4               # number of CPU cores per task 
#SBATCH --nodes=1                       # number of nodes
#SBATCH --mem-per-cpu=8000              # memory per CPU core in MB (see also --mem) 
#SBATCH --time=10:00:00                 # time limit hrs:min:sec
#SBATCH --partition=standard            # partition name (i.e. standard, windfall)
#SBATCH --account=your_account          # account name (name of your group)                     
#SBATCH --output=process-%j.out         # standard output file name (%j expands to jobID)
#SBATCH --error=process-%j.err          # standard error file name (%j expands to jobID)            
 
# --------------------------------------------------
# Load modules here
# --------------------------------------------------

# --------------------------------------------------
# Execute commands here
# --------------------------------------------------

# echo for log
echo "job started"; pwd; hostname; date

# source config gile and get sample name from input list based on job array index
source ./config.sh
export SAMPLE=`head -n +${SLURM_ARRAY_TASK_ID} $IN_LIST | tail -n 1`

# make the output directories 
TRIM_DIR="${WORK_DIR}/trimmed_reads"
UNPAIR_DIR="${WORK_DIR}/unpaired_reads"

echo "Running Trimmomatic on sample $SAMPLE"
apptainer run ${TRIMMOMATIC} trimmomatic PE -phred33 -threads 4 \
    ${FASTQ_DIR}/${SAMPLE}_R1_001.fastq.gz ${FASTQ_DIR}/${SAMPLE}_R2_001.fastq.gz \
    ${TRIM_DIR}/${SAMPLE}_R1_001.fastq.gz ${UNPAIR_DIR}/${SAMPLE}_R1_001.fastq.gz \
    ${TRIM_DIR}/${SAMPLE}_R2_001.fastq.gz ${UNPAIR_DIR}/${SAMPLE}_R2_001.fastq.gz \
    ILLUMINACLIP:TruSeq3-PE-2.fa:2:30:10 SLIDINGWINDOW:4:20 MINLEN:100 HEADCROP:15

# print date for log
echo "job ended"; date
'''

with open('01B_run_trimmomatic-slurm.sh', mode='w') as file:
    file.write(my_code)

## Step 3 QC Final Check

Create a run script that performs a final quality control check, using fastqc, on the trimmed fastq files.

This script will use the fastqc tool, but will check the reads that are in the "trimmed" directory. 

If you have any doubts about the trimming process, you can always run fastqc on the trimmed data and double check that you see all "green". You can check the fastqc files using Jupyter to check for any failures or other warnings.

In [ ]:
# Create a script to run fastqc on each of our accessions
# Round 2! This will check the fastq files after screening and cleaning with trimmomatic

my_code = '''#!/bin/bash
# --------------------------------------------------
# Request resources here
# --------------------------------------------------
#SBATCH --job-name=fastqc_trimmed       # job name
#SBATCH --ntasks=1                      # number of CPUs required 
#SBATCH --cpus-per-task=1               # number of CPU cores per task 
#SBATCH --nodes=1                       # number of nodes
#SBATCH --mem-per-cpu=4000              # memory per CPU core in MB (see also --mem) 
#SBATCH --time=10:00:00                 # time limit hrs:min:sec
#SBATCH --partition=standard            # partition name (i.e. standard, windfall)
#SBATCH --account=your_account          # account name (name of your group)                     
#SBATCH --output=process-%j.out         # standard output file name (%j expands to jobID)
#SBATCH --error=process-%j.err          # standard error file name (%j expands to jobID)                     

# --------------------------------------------------
# Load modules here
# --------------------------------------------------

# --------------------------------------------------
# Execute commands here
# --------------------------------------------------

# echo for log
echo "job started"; pwd; hostname; date

# source config gile and get sample name from input list based on job array index
source ./config.sh
export SAMPLE=`head -n +${SLURM_ARRAY_TASK_ID} $IN_LIST | tail -n 1`

# run FastQC
echo "Running FastQC on sample $SAMPLE"
apptainer run ${FASTQC} fastqc ${WORK_DIR}/trimmed_reads/${SAMPLE}_*.fastq*

# move FastQC results to a separate directory and copy to local directory for viewing
# create directory if it does not exist
TRIM_DIR="${WORK_DIR}/after_qc_trimming"
if [[ ! -d "$TRIM_DIR" ]]; then
  echo "$TRIM_DIR does not exist. Directory created"
  mkdir -p $TRIM_DIR
fi

#  move files
mv ${WORK_DIR}/trimmed_reads/${SAMPLE}_*_fastqc.html $TRIM_DIR
mv ${WORK_DIR}/trimmed_reads/${SAMPLE}_*_fastqc.zip $TRIM_DIR
cp -r $TRIM_DIR ~/01_qc_trimming

# print date for log
echo "job ended"; date

'''

with open('01C_run_fastqc-slurm.sh', mode='w') as file:
    file.write(my_code)

## Step 4: Putting it all together

Once you have created the the run scripts, you are ready to put them together in a pipeline to run each of the steps one by one. Notice which steps are dependent on the others

In [5]:
# Let's create the launcher script to kick off our pipeline.

my_code = '''#! /bin/bash

# 01A_run_fastqc: first job - no dependencies
job1=$(sbatch 01A_run_fastqc-slurm.sh)
jid1=$(echo $job1 | sed 's/^Submitted batch job //')
echo $jid1

# 01B_run_trimmomatic: jid2 depends on jid1
job2=$(sbatch --dependency=afterok:$jid1 01B_run_trimmomatic-slurm.sh)
jid2=$(echo $job2 | sed 's/^Submitted batch job //')
echo $jid2

# 01C_run_fastqc: jid3 depends on jid2
job3=$(sbatch --dependency=afterok:$jid2 01C_run_fastqc-slurm.sh)
jid3=$(echo $job3 | sed 's/^Submitted batch job //')
echo $jid3

'''

with open('01_launch_pipeline-slurm.sh', mode='w') as file:
    file.write(my_code)

In [6]:
!chmod +x *sh

In [ ]:
# now let's run it!
!./01_launch_pipeline-slurm.sh

In [ ]:
# You can check if it is running using the squeue command
# Check for all jobs under your netid
# Notice that 06B jobs are dependent on 06A jobs finishing and etc.
!squeue --user=$id

### What happens next?

Your code will take a little time to get "picked up" by the HPC and move from PD (pending) to R (running). Come back shortly to check on it. But, for now, relax and enjoy your day!

### Done